In [2]:
import os
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
from zipfile import ZipFile

# NEW CODE

In [2]:
mainurl = 'https://www.sec.gov/Archives/edgar/full-index/'
r=requests.get(mainurl)
data=r.text
soup = BeautifulSoup(data, 'lxml')

In [3]:
years = []
i = 1
while i < 200:
    years.append((str(i+1993)) + '/')
    i += 1
    
yearHyperlinks = []
newUrls = []

for i in soup.find_all('a'):
    if (i.text + '/') in years:
        yearHyperlinks.append(i)

for i in yearHyperlinks:
    newUrls.append(mainurl + str(i.get('href')))

In [4]:
quarters = ['QTR1','QTR2','QTR3','QTR4']
newNewUrls = []

for url in newUrls:
    r=requests.get(url)
    data=r.text
    soup = BeautifulSoup(data, 'lxml')
    for tag in soup.find_all('a'):
        if tag.text in quarters:
            newNewUrls.append(url + str(tag.get('href')))

In [5]:
errorCount = 1
while True:
    try:
        from io import BytesIO
        from urllib.request import urlopen
        from zipfile import ZipFile
        totalData = pd.DataFrame()
        allTheCorrectData = pd.DataFrame()
        countOfUrls = 1


        master = "master.zip"
        for url in newNewUrls:
            r=requests.get(url)
            data=r.text
            soup = BeautifulSoup(data, 'lxml')
            for tag in soup.find_all('a'):
                if tag.text == master:
                    zipurl = url + tag.text
                    with urlopen(zipurl) as zipresp:
                        with ZipFile(BytesIO(zipresp.read())) as zfile:
                            zfile.extractall('C:\\Users\\Cody\\Documents\\2017-2018 School Year\\Semester 2\\CIS440\\index files')
                            zfile.close()
            os.chdir('C:\\Users\\Cody\\Documents\\2017-2018 School Year\\Semester 2\\CIS440\\index files')
            with open('master.idx', 'rb') as f:
                data = f.read()

            data =data.decode("utf-8", errors='replace')
            data = data.split('|')
            data = data[1:]

            companies = []
            forms = []
            dateFiled = []
            hyperLink = []

            count = 0
            while count < len(data):
                companies.append(data[count])
                count+=4

            count = 1
            while count < len(data):
                forms.append(data[count])
                count+=4

            count = 2
            while count < len(data):
                dateFiled.append(data[count])
                count+=4

            count = 3
            while count < len(data):
                hyperLink.append(data[count])
                count+=4

            masterData = pd.DataFrame({'Company Names' : companies, 'Forms':forms,'Date Filed':dateFiled,'hyperLink':hyperLink})
            masterData = masterData.iloc[1:,]
            correctData = masterData[masterData['Forms'] == '10-K']                                          
            totalData = totalData.append(masterData)
            allTheCorrectData = allTheCorrectData.append(correctData)                 
            print(str(countOfUrls) + 'amount of URLs has been processed')
            countOfUrls += 1

    except ZeroDivisionError:
        print("There was an error this is number " + str(errorCount))
        errorCount += 1
        continue
    break;


1amount of URLs has been processed
2amount of URLs has been processed
3amount of URLs has been processed
4amount of URLs has been processed
5amount of URLs has been processed
6amount of URLs has been processed
7amount of URLs has been processed
8amount of URLs has been processed
9amount of URLs has been processed
10amount of URLs has been processed
11amount of URLs has been processed
12amount of URLs has been processed
13amount of URLs has been processed
14amount of URLs has been processed
15amount of URLs has been processed
16amount of URLs has been processed
17amount of URLs has been processed
18amount of URLs has been processed
19amount of URLs has been processed
20amount of URLs has been processed
21amount of URLs has been processed
22amount of URLs has been processed
23amount of URLs has been processed
24amount of URLs has been processed
25amount of URLs has been processed
26amount of URLs has been processed
27amount of URLs has been processed
28amount of URLs has been processed
2

In [6]:
correctList = []
for i in allTheCorrectData['hyperLink']:
    correctList.append(i.split('\r')[0])
dataUrls = []
for i in correctList:
    dataUrls.append('https://www.sec.gov/Archives/' + i)
    
allTheCorrectData['hyperLink'] = dataUrls

In [7]:
x = allTheCorrectData[allTheCorrectData['Date Filed'] > '2017-01-01']

In [26]:
keywords = ['ESPP','espp','Employee Stock Purchase Plan','lower of','closing period','discount','stock price on the purchase date',
            '90% of the market value']

In [13]:
os.chdir('C:\\Users\\Cody\\Documents\\2017-2018 School Year\\Semester 2\\CIS440')
# allTheCorrectData.to_csv('List of Company Documents.csv',index = False)

In [3]:
allTheCorrectData = pd.read_csv('List of Company Documents.csv')
x = allTheCorrectData[allTheCorrectData['Date Filed'] > '2017-01-01']
x[0:49]

FileNotFoundError: File b'List of Company Documents.csv' does not exist

In [31]:
textList = []
companyName = []
count2 = 0
count1 = 0
index = 0
for url in x['hyperLink']:
    r=requests.get(url)
    data=r.text
    soup = BeautifulSoup(data, 'lxml')
    keywordTF = False
    for paragraph in soup.find_all('p'):
        for z in keywords:
            if z in paragraph.text:
                textList.append(paragraph.text)
                companyName.append(x['Company Names'].iloc[index])
                count1 +=1
                print(x['Company Names'].iloc[index] + ' has a keyword which is ' + z)
                index +=1
            else:
                count2 +=1
                index += 1

    
    
print(str(count2) + ' out of ' + str(len(x['hyperLink'])) + ' dont have a keyword')
print(str(count1) + ' have keywords out of ' + str(len(x['hyperLink'])) + '\n')

CNH Equipment Trust 2013-C has a keyword which is discount
MYR GROUP INC. has a keyword which is discount
CASS INFORMATION SYSTEMS INC has a keyword which is discount
OCEANEERING INTERNATIONAL INC has a keyword which is discount
NEWELL BRANDS INC has a keyword which is discount
SUPERCONDUCTOR TECHNOLOGIES INC has a keyword which is discount
Capital Financial Holdings, Inc has a keyword which is discount


IndexError: single positional indexer is out-of-bounds

In [24]:
for url in x['hyperLink']:
    r=requests.get(url)
    data=r.text
    soup = BeautifulSoup(data, 'lxml')
    keywordTF = False
    for paragraph in soup.find_all('p'):
        for z in keywords:
            if z in paragraph.text:
                print(paragraph)

<p style="border: none; color: black; font-family: 'Times New Roman', serif; font-size: 11.0pt; margin: 0in; margin-bottom: .0001pt; margin-left: 45.0pt; text-indent: -45.0pt">10.40       Henry Schein, Inc. 2004 Employee Stock Purchase Plan, effective as of May 25, 2004. (Incorporated by reference to Exhibit D to our definitive 2004 Proxy Statement on Schedule 14A, filed on April 27, 2004.)**</p>
<p style="font: 10pt Times New Roman, Times, Serif; margin: 0pt 0; text-align: justify; text-indent: 0.5in">The following table
provides our equity compensation plan information as of December 31, 2016. Under these plans, our common stock may be issued upon
the exercise of options and purchases under our Employee Stock Purchase Plan (“ESPP”). See also the information regarding
our stock options and ESPP in Note 11 to the financial statements included herewith.</p>
<p style="font: 10pt Times New Roman, Times, Serif; margin: 0pt 0; text-align: justify; text-indent: 0.5in">The following table
pro

<p>Employee Stock Purchase Plan (ESPP) [Abstract]</p>
<p>Employee Stock Purchase Plan (ESPP) [Abstract]</p>
<p style="margin:0pt 0pt 0pt 12pt;color:#000000;font-family:Times New Roman,Times,serif;font-size: 7pt;">
<font style="display:inline;color:#000000;font-size:7pt;">Employee Stock Purchase Plan</font></p>
<p style="margin:0pt 0pt 0pt 12pt;color:#000000;font-family:Times New Roman,Times,serif;font-size: 7pt;">
<font style="display:inline;color:#000000;font-size:7pt;">Employee Stock Purchase Plan</font></p>
<p style="margin:0pt 0pt 0pt 12pt;color:#000000;font-family:Times New Roman,Times,serif;font-size: 7pt;">
<font style="display:inline;color:#000000;font-size:7pt;">Employee Stock Purchase Plan</font></p>
<p style="margin:0pt;color:#000000;font-family:Times New Roman,Times,serif;font-size: 8pt;">
<font style="display:inline;color:#000000;font-size:8pt;">Net proceeds from Class A common stock options exercised and stock issued under the Employee Stock Purchase Plan</font></p>
<p st

KeyboardInterrupt: 

0      10.40       Henry Schein, Inc. 2004 Employee S...
1      The following table\nprovides our equity compe...
2      The following table\nprovides our equity compe...
3      The Company accounts for\nstock-based compensa...
4      The expected term of\nstock options granted wa...
5      In 2013, the Company\nadopted an Employee Stoc...
6      In 2013, the Company\nadopted an Employee Stoc...
7      The ESPP is considered\ncompensatory for finan...
8      As of December 31,\n2016, which is prior to th...
9                           Employee Stock Purchase Plan
10                          Employee Stock Purchase Plan
11                          Employee Stock Purchase Plan
12     In May 2009, the Companys shareholders approv...
13     In May 2009, the Company's shareholders approv...
14        Employee Stock Purchase Plan (ESPP) [Abstract]
15        Employee Stock Purchase Plan (ESPP) [Abstract]
16                        \nEmployee Stock Purchase Plan
17                        \nEmp

In [ ]:
soup

RecursionError: maximum recursion depth exceeded while calling a Python object

In [31]:
# textList = []
# count = 0
# count1 = 0
# for i in documents:
#     keywordTF = False
#     for paragraph in i.find_all('p'):
#         for z in keywords:
#             if z in paragraph.text:
#                 textList.append(paragraph.text)
#                 keywordTF = True
#     if keywordTF == False:
#         count +=1
#     else:
#         count1 +=1

# print(str(count) + ' out of ' + str(len(documents)) + ' dont have a keyword')
# print(str(count1) + ' have keywords out of ' + str(len(documents)))

13 out of 21 dont have a keyword
8 have keywords out of 21


In [78]:
your_string = textList[0]
list_of_words = your_string.split('Employee Stock Purchase Plan')
next_word = list_of_words[list_of_words.index('Employee Stock Purchase Plan') + 1]

ValueError: 'Employee Stock Purchase Plan' is not in list

In [67]:
textList = []
count = 0
count1 = 0
for i in documents[2]:
    keywordTF = False
    for paragraph in i.find_all('p'):
        for z in keywords:
            if z in paragraph.text:
                textList.append(paragraph.text)
                keywordTF = True
    if keywordTF == False:
        count +=1
    else:
        count1 +=1

In [68]:
textList

[]

# NEW CODE IS DONE

In [62]:
textList

[]

In [59]:
len(textLis)

NameError: name 'textLis' is not defined

In [ ]:
# listOf10ks = []
# actual10Ks = []
# linkTo10K = ''
# mainurl = "https://www.sec.gov"
# count = 1

# cikList = ['0001466593', '0001121484']
# for cik in cikList:
#     url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=" + cik + "&owner=include&count=40"
#     r=requests.get(url)
#     data=r.text
#     soup = BeautifulSoup(data, 'lxml')
#     for links in soup.find_all('tr'):
#         if '10-K' in links.text:
#             for element in links.find_all('a'):
#                 if element.text == '\xa0Documents':
#                     listOf10ks = element.get('href')
#                     r=requests.get(mainurl + listOf10ks)
#                     data=r.text
#                     soup = BeautifulSoup(data, 'lxml')
#                     for links in soup.find_all('tr'):
#                         if '10-K' in links.text:
#                             for element in links.find_all('a'):
#                                 print(element.text)
#                                 linkTo10K = element.get('href')
#                                 r=requests.get(mainurl + linkTo10K)
#                                 data=r.text
#                                 soup = BeautifulSoup(data, 'lxml')
#                                 actual10Ks.append(soup)
#                                 print(str(count) + ' 10K is appended')
#                                 count +=1

In [ ]:
#     for links in soup.find_all('tr'):
#         if '10-K' in links.text:
#             for element in links.find_all('a'):
#                 if element.text == '\xa0Documents':
#                     listOf10ks = element.get('href')
#                     r=requests.get(mainurl + listOf10ks)
#                     data=r.text
#                     soup = BeautifulSoup(data, 'lxml')
#                     for links in soup.find_all('tr'):
#                         if '10-K' in links.text:
#                             for element in links.find_all('a'):
#                                 print(element.text)
#                                 linkTo10K = element.get('href')
#                                 r=requests.get(mainurl + linkTo10K)
#                                 data=r.text
#                                 soup = BeautifulSoup(data, 'lxml')
#                                 actual10Ks.append(soup)
#                                 print(str(count) + ' 10K is appended')
#                                 count +=1

In [ ]:
# os.chdir('C:\\Users\\Cody\\Documents\\2017-2018 School Year\\Semester 2\\CIS440')

In [ ]:
# with open('master.idx', 'rb') as f:
#     data = f.read()

# data =data.decode("utf-8")
# data = data.split('|')
# data = data[1:]

In [ ]:
# companies = []
# forms = []
# dateFiled = []
# hyperLink = []
# totalData = pd.DataFrame()
# allTheCorrectData = pd.DataFrame()


# data =data.decode("utf-8")
# data = data.split('|')
# data = data[1:]

# count = 0
# while count < len(data):
#     companies.append(data[count])
#     count+=4
    
# count = 1
# while count < len(data):
#     forms.append(data[count])
#     count+=4
    
# count = 2
# while count < len(data):
#     dateFiled.append(data[count])
#     count+=4
    
# count = 3
# while count < len(data):
#     hyperLink.append(data[count])
#     count+=4
    
# masterData = pd.DataFrame({'Company Names' : companies, 'Forms':forms,'Date Filed':dateFiled,'hyperLink':hyperLink})
# masterData = masterData.iloc[1:,]
# correctData = masterData[masterData['Forms'] == '10-K']

# totalData = totalData.append(masterData)
# allTheCorrectData = allTheCorrectData.append(correctData)

In [ ]:
# dataUrls = []
#                     for i in correctList:
#                         dataUrls.append('https://www.sec.gov/Archives/' + i)
                    
#                     documents = []
#                     count = 1
#                     for url in dataUrls:
#                         r=requests.get(url)
#                         data=r.text
#                         soup = BeautifulSoup(data, 'lxml')
#                         documents.append(soup)
#                         print('document' + str(count) + 'appended')
#                         count += 1 

In [ ]:
# from io import BytesIO
# from urllib.request import urlopen
# from zipfile import ZipFile

# master = "master.zip"
# r=requests.get(newNewUrls[95])
# data=r.text
# soup = BeautifulSoup(data, 'lxml')
# for tag in soup.find_all('a'):
#     if tag.text == master:
#         zipurl = newNewUrls[95] + tag.text
#         with urlopen(zipurl) as zipresp:
#             with ZipFile(BytesIO(zipresp.read())) as zfile:
#                 zfile.extractall('C:\\Users\\Cody\\Documents\\2017-2018 School Year\\Semester 2\\CIS440\\index files')
#                 zfile.close()